In [1]:
import torch # 파이토치 기본 라이브러리

# torchvision : 데이터셋, 모델 아키텍처, 컴퓨터 비전의 이미지 변환 기능 제공
from torchvision import datasets # torchvision에서 제공하는 데이터셋
from torchvision import transforms # 이미지 변환기능을 제공하는 패키지

# torch.utils.data : 파이토치 데이터 로딩 유틸리티
from torch.utils.data import DataLoader # 모델 훈련에 사용할 수 있는 미니 배치 구성하고
                                        # 매 epoch마다 데이터를 샘플링, 병렬처리 등의 일을 해주는 함수

from torch.utils.data import random_split

import numpy as np
import matplotlib.pyplot as plt

from torch.utils.tensorboard import SummaryWriter
from copy import deepcopy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [5]:
%pwd

'/content'

In [6]:
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/pj3/open.zip' './'
!unzip -q open.zip -d open/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
transform = transforms.Compose([transforms.Resize([224, 224]), transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [8]:
import pandas as pd

In [9]:
from torch.utils.data import Dataset
import glob
from PIL import Image # Image.open(path)
test_dict={'몰딩수정':0, '석고수정':1, '훼손':2, '피스':3, '녹오염':4, '가구수정':5, '오염':6, '들뜸':7, '곰팡이':8,
       '창틀,문틀수정':9, '울음':10, '오타공':11, '반점':12, '면불량':13, '터짐':14, '틈새과다':15, '걸레받이수정':16,
       '이음부불량':17, '꼬임':18}
class remodelDataset(Dataset):
    def __init__(self, root, transform):
        self.filepaths = glob.glob(root + '*/*.png')
        self.transform = transform

    def __len__(self):  # len(MyDataset)
        return len(self.filepaths)

    def __getitem__(self, index): # MyDataset[index]

        # (1) image 준비
        image_filepath = self.filepaths[index]
        image = Image.open(image_filepath)         

        
        transformed_image = self.transform(image) # Resize -> To Tensor

        # (2) label 준비
        dir_label = image_filepath.split('/')[-2]
        label_=test_dict[dir_label]
        return transformed_image ,label_
    

In [10]:
trainset = remodelDataset(root ='/content/open/train/', transform=transform)

In [11]:
all_img_list = glob.glob('./open/train/*/*.png')

In [12]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])

In [13]:
from sklearn import preprocessing

In [14]:
le = preprocessing.LabelEncoder()
df['label_num'] = le.fit_transform(df['label'])

In [15]:
df

,img_path,label,label_num
0,./open/train/녹오염/1.png,녹오염,4
1,./open/train/녹오염/11.png,녹오염,4
2,./open/train/녹오염/5.png,녹오염,4
3,./open/train/녹오염/6.png,녹오염,4
4,./open/train/녹오염/12.png,녹오염,4
...,...,...,...
3452,./open/train/훼손/413.png,훼손,18
3453,./open/train/훼손/113.png,훼손,18
3454,./open/train/훼손/954.png,훼손,18
3455,./open/train/훼손/1320.png,훼손,18


In [16]:
from sklearn.model_selection import train_test_split

train_indices, valid_indices = train_test_split(
                            range(len(df)), # X의 index
                             # y
                            stratify=df.label_num, # target의 비율이 train과 valid에 그대로 반영되게
                            test_size= 0.2, random_state=42)

In [17]:
from torch.utils.data import Subset
train_set = Subset(trainset, train_indices)
valid_set = Subset(trainset, valid_indices)

In [18]:
batch_size = 16 # 100 -> 16
# dataloader = DataLoader(데이터셋, 배치사이즈, 셔플여부.....)
trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True) # 훈련용 50000개의 데이터를 100개씩 준비
validloader = DataLoader(valid_set, batch_size=batch_size, shuffle=False) # 검증용 10000개의 데이터를 100개씩 준비

In [19]:
print(type(trainloader), len(trainloader))
print(type(validloader), len(validloader))

<class 'torch.utils.data.dataloader.DataLoader'> 173
<class 'torch.utils.data.dataloader.DataLoader'> 44


In [20]:
import torch.nn as nn # 파이토치에서 제공하는 다양한 계층 (Linear Layer, ....)
import torch.optim as optim # 옵티마이저 (경사하강법...)
import torch.nn.functional as F 

In [21]:
train_iter = iter(trainloader)
images, labels = next(train_iter)
images.shape

torch.Size([16, 3, 224, 224])

In [22]:
labels

tensor([ 6,  2, 13,  0,  2,  2, 13,  6,  2,  8, 16,  6, 16,  2,  2, 14])

In [ ]:
conv_block5 = nn.Sequential(
                                      nn.Conv2d(in_channels=3, out_channels=256, kernel_size=3, stride=1, padding=1),
                                      nn.BatchNorm2d(num_features=256),
                                      nn.Dropout(0.1), 
                                      nn.ReLU(),   
                                      nn.MaxPool2d(kernel_size=3, stride=2)                                   
                                     )
conv_block5_out = conv_block5(images)                                     
conv_block5_out.shape

In [23]:
class AlexNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_block1 = nn.Sequential(
                                      nn.Conv2d(in_channels=3, out_channels=96, kernel_size=3, stride=1, padding=1),
                                      nn.BatchNorm2d(num_features=96),
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=3, stride=2)
                                     ) # [16, 96, 111, 111]
    self.conv_block2 = nn.Sequential(
                                      nn.Conv2d(in_channels=96, out_channels=256, kernel_size=3, stride=1, padding=1),
                                      nn.BatchNorm2d(num_features=256),                                      
                                      nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=3, stride=2)
                                     ) # [16, 256, 111, 111]

    self.conv_block3 = nn.Sequential(
                                      nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
                                      nn.BatchNorm2d(num_features=384), 
                                      nn.Dropout(0.1),                                    
                                      nn.ReLU(),                                      
                                     ) # [16, 384, 224, 224]     

    self.conv_block4 = nn.Sequential(
                                      nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
                                      nn.BatchNorm2d(num_features=384),  
                                      nn.Dropout(0.3),                                    
                                      nn.ReLU(),                                      
                                     ) # [16, 384, 224, 224]   

    self.conv_block5 = nn.Sequential(
                                      nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
                                      nn.BatchNorm2d(num_features=256),
                                      nn.Dropout(0.1), 
                                      nn.ReLU(),   
                                      nn.MaxPool2d(kernel_size=3, stride=2)                                   
                                     ) # [16, 256, 3, 3]                                                                                                       

    self.linear1 = nn.Linear(in_features=256*3*3, out_features=512)
    self.batch_norm = nn.BatchNorm1d(num_features=512)
    self.linear2 = nn.Linear(in_features=512, out_features=19)

  def forward(self, x):
    x = self.conv_block1(x) 
    x = self.conv_block2(x) 
    x = self.conv_block3(x) 
    x = self.conv_block4(x) 
    x = self.conv_block5(x) 
    
    # reshape할 형상 : (batch_size x 256*3*3)
    # x = x.view(-1, 256*3*3) # option 1 : view
    x = torch.flatten(x, 1) # option 2 : flatten 
    # x = x.reshape(x.shape[0], -1) # option 3 : reshape
    x = F.dropout(x, 0.3)
    x = self.linear1(x)
    x = self.batch_norm(x)
    x = F.dropout(x, 0.1)
    x = F.relu(x)
    x = self.linear2(x)
    return x

In [24]:
model = AlexNet()
model.to(device)
model

AlexNet(
  (conv_block1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.1, inplace=False)
    (3): ReLU()
  )
  (conv_block4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(384, eps=1e-05

In [25]:
learning_rate = 0.001
# 손실함수
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저(최적화함수, 예:경사하강법)
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 규제의 강도 설정 weight_decay
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)

# Learning Rate Schedule
# https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html

# 모니터링하고 있는 값(예:valid_loss)의 최소값(min) 또는 최대값(max) patience 기간동안 줄어들지 않을 때(OnPlateau) lr에 factor(0.1)를 곱해주는 전략
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4, verbose=True)

In [26]:
from torchsummary import summary

In [27]:
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 32, 32]           2,688
       BatchNorm2d-2           [-1, 96, 32, 32]             192
              ReLU-3           [-1, 96, 32, 32]               0
         MaxPool2d-4           [-1, 96, 15, 15]               0
            Conv2d-5          [-1, 256, 15, 15]         221,440
       BatchNorm2d-6          [-1, 256, 15, 15]             512
              ReLU-7          [-1, 256, 15, 15]               0
         MaxPool2d-8            [-1, 256, 7, 7]               0
            Conv2d-9            [-1, 384, 7, 7]         885,120
      BatchNorm2d-10            [-1, 384, 7, 7]             768
          Dropout-11            [-1, 384, 7, 7]               0
             ReLU-12            [-1, 384, 7, 7]               0
           Conv2d-13            [-1, 384, 7, 7]       1,327,488
      BatchNorm2d-14            [-1, 38

In [28]:
def validate(model, validloader, loss_fn):
  total = 0   
  correct = 0
  valid_loss = 0
  valid_accuracy = 0

  # 전방향 예측을 구할 때는 gradient가 필요가 없음음
  with torch.no_grad():
    for images, labels in validloader: # 이터레이터로부터 next()가 호출되며 미니배치 100개씩을 반환(images, labels)      
      # images, labels : (torch.Size([16, 3, 32, 32]), torch.Size([16]))
      # 0. Data를 GPU로 보내기
      images, labels = images.to(device), labels.to(device)

      # 1. 입력 데이터 준비
      # not Flatten !!
      # images.resize_(images.size()[0], 784)

      # 2. 전방향(Forward) 예측
      logit = model(images) # 예측 점수
      _, preds = torch.max(logit, 1) # 배치에 대한 최종 예측
      # preds = logit.max(dim=1)[1] 
      correct += int((preds == labels).sum()) # 배치 중 맞은 것의 개수가 correct에 누적
      total += labels.shape[0] # 배치 사이즈만큼씩 total에 누적

      loss = loss_fn(logit, labels)
      valid_loss += loss.item() # tensor에서 값을 꺼내와서, 배치의 loss 평균값을 valid_loss에 누적

    valid_accuracy = correct / total
  
  return valid_loss, valid_accuracy

In [29]:
writer = SummaryWriter()

def train_loop(model, trainloader, loss_fn, epochs, optimizer):  
  steps = 0
  steps_per_epoch = len(trainloader) 
  min_loss = 1000000
  max_accuracy = 0
  trigger = 0
  patience = 7 

  for epoch in range(epochs):
    model.train() # 훈련 모드
    train_loss = 0
    for images, labels in trainloader: # 이터레이터로부터 next()가 호출되며 미니배치를 반환(images, labels)
      steps += 1
      # images, labels : (torch.Size([16, 3, 32, 32]), torch.Size([16]))
      # 0. Data를 GPU로 보내기
      images, labels = images.to(device), labels.to(device)

      # 1. 입력 데이터 준비
      # not Flatten !!
      # images.resize_(images.shape[0], 784) 

      # 2. 전방향(forward) 예측
      predict = model(images) # 예측 점수
      loss = loss_fn(predict, labels) # 예측 점수와 정답을 CrossEntropyLoss에 넣어 Loss값 반환

      # 3. 역방향(backward) 오차(Gradient) 전파
      optimizer.zero_grad() # Gradient가 누적되지 않게 하기 위해
      loss.backward() # 모델파리미터들의 Gradient 전파

      # 4. 경사 하강법으로 모델 파라미터 업데이트
      optimizer.step() # W <- W -lr*Gradient

      train_loss += loss.item()
      if (steps % steps_per_epoch) == 0 : 
        model.eval() # 평가 모드 : 평가에서 사용하지 않을 계층(배치 정규화, 드롭아웃)들을 수행하지 않게 하기 위해서
        valid_loss, valid_accuracy = validate(model, validloader, loss_fn)

        # tensorboard 시각화를 위한 로그 이벤트 등록
        writer.add_scalar('Train Loss', train_loss/len(trainloader), epoch+1)
        writer.add_scalar('Valid Loss', valid_loss/len(validloader), epoch+1)
        writer.add_scalars('Train Loss and Valid Loss',
                          {'Train' : train_loss/len(trainloader),
                            'Valid' : valid_loss/len(validloader)}, epoch+1)
        writer.add_scalar('Valid Accuracy', valid_accuracy, epoch+1)
        # -------------------------------------------

        print('Epoch : {}/{}.......'.format(epoch+1, epochs),            
              'Train Loss : {:.3f}'.format(train_loss/len(trainloader)), 
              'Valid Loss : {:.3f}'.format(valid_loss/len(validloader)), 
              'Valid Accuracy : {:.3f}'.format(valid_accuracy)            
              )
        
        # Best model 저장    
        # option 1 : valid_loss 모니터링
        # if valid_loss < min_loss: # 바로 이전 epoch의 loss보다 작으면 저장하기
        #   min_loss = valid_loss
        #   best_model_state = deepcopy(model.state_dict())          
        #   torch.save(best_model_state, 'best_checkpoint.pth')     
        
        # option 2 : valid_accuracy 모니터링      
        if valid_accuracy > max_accuracy : # 바로 이전 epoch의 accuracy보다 크면 저장하기
          max_accuracy = valid_accuracy
          best_model_state = deepcopy(model.state_dict())          
          torch.save(best_model_state, 'best_checkpoint.pth')  
        # -------------------------------------------

        # Early Stopping (조기 종료)
        if valid_loss > min_loss: # valid_loss가 min_loss를 갱신하지 못하면
          trigger += 1
          print('trigger : ', trigger)
          if trigger > patience:
            print('Early Stopping !!!')
            print('Training loop is finished !!')
            writer.flush()   
            return
        else:
          trigger = 0
          min_loss = valid_loss
        # -------------------------------------------

        # Learning Rate Scheduler
        scheduler.step(valid_loss)
        # -------------------------------------------
        
  writer.flush()
  return  

In [30]:
epochs = 55
%time train_loop(model, trainloader, loss_fn, epochs, optimizer)
writer.close()

RuntimeError: ignored